`BcForms` is a toolkit for concretely describing the primary structure of macromolecular complexes, including non-canonical monomeric forms and intra and inter-subunit crosslinks. `BcForms` includes a textual grammar for describing complexes and a Python library, a command line program, and a REST API for validating and manipulating complexes described in this grammar.
`BcForms` represents complexes as sets of subunits, with their stoichiometries, and covalent crosslinks which link the subunits.
DNA, RNA, and protein subunits can be represented using `BpForms`. Small molecule subunits can be represented using `openbabel.OBMol`, typically imported from SMILES or InChI.
This Jupyter notebook illustrates how to use the `BcForms` Python library via some simple and complex examples. Please see the [documentation](https://docs.karrlab.org/bcforms/) for more information.

## Import `BcForms` to represent complexes and other modules to represent subunits

In [1]:
import bcforms # represent complexes
import bpforms # represent DNA, RNA, and protein subunits
import openbabel # represent small molecule subunits and import their descriptions from SMILES, InChI, etc.

## Simple `BcForms` examples
This section illustrates how to use the `BcForms` grammar with several simple examples.

### Representing, validating, and calculating properties of a homodimer with no crosslinks
#### Representing a homodimer with no crosslinks
This example illustrates how to use `BcForms` to describe a homodimer of subunit `unit_1`, which is the peptide tri-alanine (AAA).

In [2]:
bc_form_1 = bcforms.BcForm().from_str('2 * unit_1')
bc_form_1.set_subunit_attribute('unit_1', 'structure', bpforms.ProteinForm().from_str('AAA'))

#### Validate the `BcForm` representation of the homodimer
The `BcForm.validate` method can be used to check that there are no errors in the `BcForms` representation of the homodimer.
The `validate` method will return a list of any errors.

In [3]:
bc_form_1.validate()

[]

#### Calculate physical properties and structure of the `Bcforms` object
`BcForms` can also be used to calculate properties of complexes, such as their atom-bond structure, formula, molecular weight, and charge.

In [4]:
str(bc_form_1.get_formula())

'C18H36N6O8'

In [5]:
bc_form_1.get_mol_wt()

464.52000000000004

In [6]:
bc_form_1.get_charge()

2

In [7]:
bc_form_1.export()

'C[C@H]([NH3+])C(=O)N[C@@H](C)C(=O)N[C@@H](C)C(=O)O.C[C@H]([NH3+])C(=O)N[C@@H](C)C(=O)N[C@@H](C)C(=O)O'

#### Representing a complex when concrete structure is unknown
Sometimes, the detailed structure of the subunits are not known. In that case, `BcForms` can still represent the complexes and hold the known information such as formula, molecular weight, and charges.

In [8]:
bc_form_1_u = bcforms.BcForm().from_str('2 * unit_u')
bc_form_1_u.set_subunit_attribute('unit_u', 'formula', 'C9H18N3O4')

#### Calculate physical properties of the `Bcforms` object when structure is unknown
When structure of subunits is not known, if formula is known, `BcForms` can calculate the formula and molecular weight of the complex. If only molecular weight or charge of subunits is known, then `BcForms` can only calculate the respective properties of the complex.

In [9]:
str(bc_form_1_u.get_formula())

'C18H36N6O8'

In [10]:
bc_form_1_u.get_mol_wt()

464.52000000000004

#### Check if two biocomplexes are equal
The `BcForm.is_equal` method can be used to check whether two complexes are semantically equal.

In [11]:
bc_form_1_t = bcforms.BcForm().from_str('unit_1 + unit_1')
bc_form_1_t.set_subunit_attribute('unit_1', 'structure', bpforms.ProteinForm().from_str('AAA'))
bc_form_1.is_equal(bc_form_1_t)

True

In [12]:
bc_form_1_f = bcforms.BcForm().from_str('3 * unit_1')
bc_form_1_f.set_subunit_attribute('unit_1', 'structure', bpforms.ProteinForm().from_str('AAA'))
bc_form_1.is_equal(bc_form_1_f)

False

### Representing, validating, and calculating properties of a homodimer with a disulfide bond with inline definition
This example illustrates how to use `BcForms` to represent a homodimer of subunit `unit_2`, which is the single amino acid cysteine, with a disulfide bond between the cysteines. The crosslink can be either defined inline or described using our ontology of crosslinks. This example illustrates how to define a disulfide bond inline.

In [13]:
bc_form_2_a = bcforms.BcForm().from_str('2 * unit_2'
                                      '| x-link: [ l-bond-atom: unit_2(1)-1S11 |'
                                                 ' l-displaced-atom: unit_2(1)-1H11 |'
                                                 ' r-bond-atom: unit_2(2)-1S11 |'
                                                 ' r-displaced-atom: unit_2(2)-1H11 ]')
bc_form_2_a.set_subunit_attribute('unit_2', 'structure', bpforms.ProteinForm().from_str('C'))

#### Validate the `BcForms` representation of the homodimer

In [14]:
bc_form_2_a.validate()

[]

#### Calculate physical properties and structure of the `Bcforms` object

In [15]:
str(bc_form_2_a.get_formula())

'C6H14N2O4S2'

In [16]:
bc_form_2_a.get_mol_wt()

242.308

In [17]:
bc_form_2_a.get_charge()

2

In [18]:
bc_form_2_a.export()

'OC(=O)[C@@H]([NH3+])CSSC[C@@H](C(=O)O)[NH3+]'

### Representing, validating, and calculating properties of a homodimer with a disulfide bond with our ontology of crosslinks
Alternatively, crosslinks can be described using our ontology of crosslinks. This enables more compact descriptions of complexes. This example illustrates how to use our ontology to describe the same complex. The list of crosslinks defined in the ontology is available at [https://www.bpforms.org/crosslinks.html](https://www.bpforms.org).

In [19]:
bc_form_2_b = bcforms.BcForm().from_str('2 * unit_2'
                                      '| x-link: [ type: disulfide |'
                                                 ' l-monomer: unit_2(1)-1 |'
                                                 ' r-monomer: unit_2(2)-1 ]')
bc_form_2_b.set_subunit_attribute('unit_2', 'structure', bpforms.ProteinForm().from_str('C'))

#### Validate the `BcForms` representation of the homodimer

In [20]:
bc_form_2_b.validate()

[]

#### Calculate physical properties and structure of the `Bcforms` object

In [21]:
str(bc_form_2_b.get_formula())

'C6H14N2O4S2'

In [22]:
bc_form_2_b.get_mol_wt()

242.308

In [23]:
bc_form_2_b.get_charge()

2

In [24]:
bc_form_2_b.export()

'OC(=O)[C@@H]([NH3+])CSSC[C@@H](C(=O)O)[NH3+]'

## Examples of real macromolecular complexes
This section illustrates how to use `BcForms` to represent real protein complexes that are annotated in the PDB and UniProt.

### Protein complexes with disulfide bonds
Disulfide bonds are one of the best characterized types of intermolecular crosslinks. Disulfide bonds are covalent bonds between cysteine residues in proteins. For more information, see [UniProt](https://www.uniprot.org/help/disulfid).
`BpForms` can be used to represent intrachain disulfide bonds, and `BcForms` can be used to represent interchain disulfide bonds.
Here, we demonstrate how to use `BcForms` to represent disulfide bonds via three examples: a parallel homodimer, an anti-parallel homodimer, and a heterodimer.

#### Parallel homodimer
This example illustrates how to use `BcForms` to represent bone morphogenetic protein 2-A (bmp2-a, [P25703](https://www.uniprot.org/uniprot/P25703)) from *Xenopus laevis*, which forms a parallel homodimer with a disulfide link at C-362 after post-translational processing to remove all of the amino acids before 285th residue and after the 398th residue.
##### Representing and calculating properties of the subunits
First, we illustrate how to use `BpForms` to represent the post-translationally processed subunits.

In [25]:
def get_chain(fasta, chain_idx):
    chain_fasta = fasta[chain_idx[0] - 1:chain_idx[1]]
    chain = bpforms.ProteinForm().from_str(chain_fasta)
    return chain

In [26]:
p25703_fasta = ('MVAGIHSLLLLLFYQVLLSGCTGLIPEEGKRKYTESGRSSPQQSQRVLNQFELRLLSMFG'
                'LKRRPTPGKNVVIPPYMLDLYHLHLAQLAADEGTSAMDFQMERAASRANTVRSFHHEESM'
                'EEIPESREKTIQRFFFNLSSIPNEELVTSAELRIFREQVQEPFESDSSKLHRINIYDIVK'
                'PAAAASRGPVVRLLDTRLVHHNESKWESFDVTPAIARWIAHKQPNHGFVVEVNHLDNDKN'
                'VPKKHVRISRSLTPDKDNWPQIRPLLVTFSHDGKGHALHKRQKRQARHKQRKRLKSSCRR'
                'HPLYVDFSDVGWNDWIVAPPGYHAFYCHGECPFPLADHLNSTNHAIVQTLVNSVNTNIPK'
                'ACCVPTELSAISMLYLDENEKVVLKNYQDMVVEGCGCR')
p25703_chain_idx = (285, 398)
p25703_chain = get_chain(p25703_fasta, p25703_chain_idx)
assert len(p25703_chain) == p25703_chain_idx[1] - p25703_chain_idx[0] + 1

In [27]:
str(p25703_chain.get_formula())

'C570H895N164O154S9'

In [28]:
p25703_chain.get_mol_wt()

12797.964

##### Representing and calculating properties of the complex
Second, we illustrate how to use `BcForms` to represent the complex.

In [29]:
assert str(p25703_chain)[362 - p25703_chain_idx[0] + 1 - 1] == p25703_fasta[362-1]
str_bmp2a = ('2 * p25703'
             ' | x-link: [ l-bond-atom: p25703(1)-{}S11 |'
                         ' r-bond-atom: p25703(2)-{}S11 |' 
                         ' l-displaced-atom: p25703(1)-{}H11 |'
                         ' r-displaced-atom: p25703(2)-{}H11 ]'.format( \
                           362 - p25703_chain_idx[0] + 1, 362 - p25703_chain_idx[0] + 1, \
                           362 - p25703_chain_idx[0] + 1, 362 - p25703_chain_idx[0] + 1))
bc_form_bmp2a = bcforms.BcForm().from_str(str_bmp2a)
bc_form_bmp2a.set_subunit_attribute('p25703', 'structure', p25703_chain)

With this `BcForms`, we can calculate properties of the complex such as its formula, molecular weight, and charge.

In [30]:
str(bc_form_bmp2a.get_formula())

'C1140H1788N328O308S18'

In [31]:
bc_form_bmp2a.get_mol_wt()

25593.911999999997

As shown above, the formula of the complex is 2 hydrogen atoms less than 2 times the formula of the subunit, and the molecular weight is approximately 2 Da less than 2 times that of the subunit.

#### Anti-parallel homodimer
This following example illustrates how `BcForms` can be used to represent anti-parallel homodimer of disintegrin schistatin ([P83658](https://www.uniprot.org/uniprot/P83658)) from *Echis carinatus*, with the subunits linked by disulfide bonds between C-7 and C-12.
##### Representing the subunits with `BpForms`
First, we illustrate how `BpForms` can be used to represent the structure of the subunits and calculate properties of the subunits.

In [32]:
p83658_fasta = ('NSVHPCCDPVICEPREGEHCISGPCCENCYFLNSGTICKRARGDGNQDYCTGITPDCPRN'
                'RYNV')
p83658_chain_idx = (1, 64)
p83658_chain = get_chain(p83658_fasta, p83658_chain_idx)

In [33]:
str(p83658_chain.get_formula())

'C290H456N91O89S10'

In [34]:
p83658_chain.get_mol_wt()

6961.986

##### Representing the complex with `BcForms`
Second, we illustrate how `BcForms` can be used to represent the complex.

In [35]:
str_dids = ('2 * p83658'
            ' | x-link: [ l-bond-atom: p83658(1)-{}S11 |'
                        ' r-bond-atom: p83658(2)-{}S11 |'
                        ' l-displaced-atom: p83658(1)-{}H11 |'
                        ' r-displaced-atom: p83658(2)-{}H11 ]'
            ' | x-link: [ l-bond-atom: p83658(1)-{}S11 |'
                        ' r-bond-atom: p83658(2)-{}S11 |'
                        ' l-displaced-atom: p83658(1)-{}H11 |'
                        ' r-displaced-atom: p83658(2)-{}H11 ]'.format( \
                          7 - p83658_chain_idx[0] + 1, 12 - p83658_chain_idx[0] + 1, \
                          7 - p83658_chain_idx[0] + 1, 12 - p83658_chain_idx[0] + 1, \
                          7 - p83658_chain_idx[0] + 1, 12 - p83658_chain_idx[0] + 1, \
                          7 - p83658_chain_idx[0] + 1, 12 - p83658_chain_idx[0] + 1))
bc_form_dids = bcforms.BcForm().from_str(str_dids)
bc_form_dids.set_subunit_attribute('p83658', 'structure', p83658_chain)

As illustrated below, `BcForms` can also be used to compute physical properties of the complex.

In [36]:
str(bc_form_dids.get_formula())

'C580H908N182O178S20'

In [37]:
bc_form_dids.get_mol_wt()

13919.94

As shown above, the formula of the complex is 4 hydrogens less than 2 times the formula of the subunit, and the molecular weight of the complex is approximately 4 Da less than 2 times that of the subunit.

#### Heterodimer
Here, we illustrate how `BcForms` can be used to represent snaclec botrocetin of *Bothrops jajaraca*, a heterodimer of subunits alpha ([P22029](https://www.uniprot.org/uniprot/P22029)) and beta ([P22030](https://www.uniprot.org/uniprot/P22030)) which are linked by a disulfide bond between C-80 of P22029 and C-75 of P22030.
##### Representing the subunits with `BpForms`
First, we illustrate how `BpForms` can be used to represent and calculate properties of the two subunits.

In [38]:
p22029_fasta = ('DCPSGWSSYEGNCYKFFQQKMNWADAERFCSEQAKGGHLVSIKIYSKEKDFVGDLVTKNI'
                'QSSDLYAWIGLRVENKEKQCSSEWSDGSSVSYENVVERTVKKCFALEKDLGFVLWINLYC'
                'AQKNPFVCKSPPP')
p22029_chain_idx = (1, 133)
p22029_chain = get_chain(p22029_fasta, p22029_chain_idx)

In [39]:
str(p22029_chain.get_formula())

'C682H1048N176O187S8'

In [40]:
p22029_chain.get_mol_wt()

14961.410999999998

In [41]:
p22030_fasta = ('DCPPDWSSYEGHCYRFFKEWMHWDDAEEFCTEQQTGAHLVSFQSKEEADFVRSLTSEMLK'
                'GDVVWIGLSDVWNKCRFEWTDGMEFDYDDYYLIAEYECVASKPTNNKWWIIPCTRFKNFV'
                'CEFQA')
p22030_chain_idx = (1, 125)
p22030_chain = get_chain(p22030_fasta, p22030_chain_idx)

In [42]:
str(p22030_chain.get_formula())

'C682H972N166O178S10'

In [43]:
p22030_chain.get_mol_wt()

14664.862000000001

##### Representing the complex with `BcForms`
Below, we illustrate how to use `BcForms` to represent the heterodimer.

In [44]:
str_sle = ('p22029 + p22030'
           ' | x-link: [ l-bond-atom: p22029(1)-{}S11 |'
                       ' r-bond-atom: p22030(1)-{}S11 |'
                       ' l-displaced-atom: p22029(1)-{}H11 |'
                       ' r-displaced-atom: p22030(1)-{}H11 ]'.format( \
                         80 - p22029_chain_idx[0] + 1, 75 - p22030_chain_idx[0] + 1, \
                         80 - p22029_chain_idx[0] + 1, 75 - p22030_chain_idx[0] + 1))
bc_form_sle = bcforms.BcForm().from_str(str_sle)
bc_form_sle.set_subunit_attribute('p22029', 'structure', p22029_chain)
bc_form_sle.set_subunit_attribute('p22030', 'structure', p22030_chain)

As illustrated below, `BcForms` can also be used to compute the formula and molecular weight of the complex.

In [45]:
str(bc_form_sle.get_formula())

'C1364H2018N342O365S18'

In [46]:
bc_form_sle.get_mol_wt()

29624.256999999998

### Protein complexes with other types of crosslinks
DNA, RNA, and protein polymers can be covalent bound by a variety of other types of crosslinks in addition to disulfide bonds. See [UniProt](https://www.uniprot.org/help/crosslnk) for more information.
The PDB, UniProt, and other database contain detailed information about crosslinks among DNA, RNA, proteins, and small molecules.
Here, we illustrate how `BcForms` can be used to concretely represent complexes that contain several types of these crosslinks.

#### Sumoylation
Sumoylation is a common interchain crosslink that binds small SUMO peptides to proteins to retain proteins in the nucleus.
In this example, we illustrate how `BcForms` can be used to represent the sumoylation of human chromatin assembly factor 1 subunit A ([Q13111](https://www.uniprot.org/uniprot/Q13111)) by [P63165](https://www.uniprot.org/uniprot/P63165). The sumoylation takes place between K-182 of Q13111 and G-97 of P63165, forming a Glycyl lysine isopeptide bond ([AA0125](https://annotation.dbi.udel.edu/cgi-bin/resid?id=AA0125)).
##### Representing the subunits with `BpForms`
First, we illustrate how `BpForms` can be used to describe the subunits.

In [47]:
q13111_fasta = ('MLEELECGAPGARGAATAMDCKDRPAFPVKKLIQARLPFKRLNLVPKGKADDMSDDQGTS'
                'VQSKSPDLEASLDTLENNCHVGSDIDFRPKLVNGKGPLDNFLRNRIETSIGQSTVIIDLT'
                'EDSNEQPDSLVDHNKLNSEASPSREAINGQREDTGDQQGLLKAIQNDKLAFPGETLSDIP'
                'CKTEEEGVGCGGAGRRGDSQECSPRSCPELTSGPRMCPRKEQDSWSEAGGILFKGKVPMV'
                'VLQDILAVRPPQIKSLPATPQGKNMTPESEVLESFPEEDSVLSHSSLSSPSSTSSPEGPP'
                'APPKQHSSTSPFPTSTPLRRITKKFVKGSTEKNKLRLQRDQERLGKQLKLRAEREEKEKL'
                'KEEAKRAKEEAKKKKEEEKELKEKERREKREKDEKEKAEKQRLKEERRKERQEALEAKLE'
                'EKRKKEEEKRLREEEKRIKAEKAEITRFFQKPKTPQAPKTLAGSCGKFAPFEIKEHMVLA'
                'PRRRTAFHPDLCSQLDQLLQQQSGEFSFLKDLKGRQPLRSGPTHVSTRNADIFNSDVVIV'
                'ERGKGDGVPERRKFGRMKLLQFCENHRPAYWGTWNKKTALIRARDPWAQDTKLLDYEVDS'
                'DEEWEEEEPGESLSHSEGDDDDDMGEDEDEDDGFFVPHGYLSEDEGVTEECADPENHKVR'
                'QKLKAKEWDEFLAKGKRFRVLQPVKIGCVWAADRDCAGDDLKVLQQFAACFLETLPAQEE'
                'QTPKASKRERRDEQILAQLLPLLHGNVNGSKVIIREFQEHCRRGLLSNHTGSPRSPSTTY'
                'LHTPTPSEDAAIPSKSRLKRLISENSVYEKRPDFRMCWYVHPQVLQSFQQEHLPVPCQWS'
                'YVTSVPSAPKEDSGSVPSTGPSQGTPISLKRKSAGSMCITQFMKKRRHDGQIGAEDMDGF'
                'QADTEEEEEEEGDCMIVDVPDAAEVQAPCGAASGAGGGVGVDTGKATLTASPLGAS')
q13111_chain_idx = (1, 956)
q13111_chain = get_chain(q13111_fasta, q13111_chain_idx)

In [48]:
str(q13111_chain.get_formula())

'C4613H7563N1356O1324S35'

In [49]:
q13111_chain.get_mol_wt()

104328.51500000001

In [50]:
p63165_fasta = ('MSDQEAKPSTEDLGDKKEGEYIKLKVIGQDSSEIHFKVKMTTHLKKLKESYCQRQGVPMN'
                'SLRFLFEGQRIADNHTPKELGMEEEDVIEVYQEQTGGHSTV')
p63165_chain_idx = (2, 97)
p63165_chain = get_chain(p63165_fasta, p63165_chain_idx)

In [51]:
str(p63165_chain.get_formula())

'C478H778N131O139S4'

In [52]:
p63165_chain.get_mol_wt()

10712.499999999998

##### Representing the complex with `BcForms`
Second, we illustrate how `BcForms` can be used to represent the complex.

In [53]:
str_sumo_a = ('q13111 + p63165'
           ' | x-link: [ l-bond-atom: q13111-{}N1-1 |'
                       ' r-bond-atom: p63165-{}C2 |'
                       ' l-displaced-atom: q13111-{}H1+1 |'
                       ' l-displaced-atom: q13111-{}H1 |' 
                       ' r-displaced-atom: p63165-{}O1 |'
                       ' r-displaced-atom: p63165-{}H1 ]'.format( \
                         182 - q13111_chain_idx[0] + 1, 97 - p63165_chain_idx[0] + 1, \
                         182 - q13111_chain_idx[0] + 1, 182 - q13111_chain_idx[0] + 1, \
                         97 - p63165_chain_idx[0] + 1, 97 - p63165_chain_idx[0] + 1))
bc_form_sumo_a = bcforms.BcForm().from_str(str_sumo_a)
bc_form_sumo_a.set_subunit_attribute('q13111', 'structure', q13111_chain)
bc_form_sumo_a.set_subunit_attribute('p63165', 'structure', p63165_chain)

In [54]:
str(bc_form_sumo_a.get_formula())

'C5091H8338N1487O1462S39'

In [55]:
bc_form_sumo_a.get_mol_wt()

115021.99200000001

##### Representing the complex with our ontology of crosslinks
Alternatively, the glycyl lysine isopepteide bond can be described using our ontology of crosslinks.

In [56]:
str_sumo_b = ('p63165 + q13111'
           ' | x-link: [ type: glycyl_lysine_isopeptide |'
                       ' l-monomer: p63165-{} |'
                       ' r-monomer: q13111-{} ]'.format( \
                         97 - p63165_chain_idx[0] + 1, 182 - q13111_chain_idx[0] + 1))
bc_form_sumo_b = bcforms.BcForm().from_str(str_sumo_b)
bc_form_sumo_b.set_subunit_attribute('q13111', 'structure', q13111_chain)
bc_form_sumo_b.set_subunit_attribute('p63165', 'structure', p63165_chain)

In [57]:
str(bc_form_sumo_b.get_formula())

'C5091H8338N1487O1462S39'

In [58]:
bc_form_sumo_b.get_mol_wt()

115021.99200000001

#### Pupylation
Proteins can be tagged for proteasomal degradation through the formation of crosslinks with pup protein tags.
This example illustrates how `BcForms` can be used to represent the pupylation of 10 kDa chaperonin ([P9WPE5](https://www.uniprot.org/uniprot/P9WPE5)) of *Mycobacterium tuberculosis* by prokaryotic ubiquitin-like protein Pup ([P9WHN5](https://www.uniprot.org/uniprot/P9WHN5)), involving a isoglutamyl lysine isopeptide bond ([AA0124](https://annotation.dbi.udel.edu/cgi-bin/resid?id=AA0124)).
##### Representing the subunits with `BpForms`
First, we illustrate how `BpForms` can be used to represent the subunits.

In [59]:
P9WHN5_fasta = ('MAQEQTKRGGGGGDDDDIAGSTAAGQERREKLTEETDDLLDEIDDVLEENAEDFVRAYVQ'
                'KGGQ')
P9WHN5_chain_idx = (1, 64)
P9WHN5_chain = get_chain(P9WHN5_fasta, P9WHN5_chain_idx)

In [60]:
str(P9WHN5_chain.get_formula())

'C285H463N85O96S'

In [61]:
P9WHN5_chain.get_mol_wt()

6648.398

In [62]:
P9WPE5_fasta = ('MAKVNIKPLEDKILVQANEAETTTASGLVIPDTAKEKPQEGTVVAVGPGRWDEDGEKRIP'
                'LDVAEGDTVIYSKYGGTEIKYNGEEYLILSARDVLAVVSK')
P9WPE5_chain_idx = (2, 100)
P9WPE5_chain = get_chain(P9WPE5_fasta, P9WPE5_chain_idx)

In [63]:
str(P9WPE5_chain.get_formula())

'C473H780N123O138'

In [64]:
P9WPE5_chain.get_mol_wt()

10398.166

##### Representing the complex with `BcForms`
Second, we illusrate how `BcForms` can be used to represent and calculate properties of the complex.

In [65]:
str_pup = ('p9whn5 + p9wpe5'
           ' | x-link: [ l-bond-atom: p9whn5-{}N1-1 |'
                       ' r-bond-atom: p9wpe5-{}C2 |'
                       ' l-displaced-atom: p9whn5-{}H1+1 |'
                       ' l-displaced-atom: p9whn5-{}H1 |' 
                       ' r-displaced-atom: p9wpe5-{}N1 |'
                       ' r-displaced-atom: p9wpe5-{}H1 |'
                       ' r-displaced-atom: p9wpe5-{}H1 ]'.format( \
                         100 - P9WHN5_chain_idx[0] + 1, 64 - P9WPE5_chain_idx[0] + 1, \
                         100 - P9WHN5_chain_idx[0] + 1, 100 - P9WHN5_chain_idx[0] + 1, \
                         64 - P9WPE5_chain_idx[0] + 1, 64 - P9WPE5_chain_idx[0] + 1, \
                         64 - P9WPE5_chain_idx[0] + 1))
bc_form_pup = bcforms.BcForm().from_str(str_pup)
bc_form_pup.set_subunit_attribute('p9whn5', 'structure', P9WHN5_chain)
bc_form_pup.set_subunit_attribute('p9wpe5', 'structure', P9WPE5_chain)

In [66]:
str(bc_form_pup.get_formula())

'C758H1239N207O234S'

In [67]:
bc_form_pup.get_mol_wt()

17028.52499999999

### Protein-cofactor/coenzyme complexes
Some enzymes require certain non-protein substances to function properly. These non-protein compoenents of the enzyme complexes can be either inorganic or organic. The inorganic components, such as metal ions, are often termed cofactors, while the organic components, such as certain vitamins, are often termed coenzymes. For more information, see [Uniprot](https://www.uniprot.org/help/cofactor). `BcForms` is capable of representing complexes that consist of protein and non-protein components.

#### Protein-cofactor complexes
In this example, we demonstrate the `BcForms` representation of a simple protein-cofactor complex ubiquinol oxidase 1a, mitochondrial ([Q39219](https://www.uniprot.org/uniprot/Q39219)) in *Arabidopsis thaliana*. The protein can bind two iron ions.

##### Representing the subunits with `BpForms`
First, we represent the protein subunit Q39219 using `BpForms`.

In [68]:
Q39219_fasta = ('MMITRGGAKAAKSLLVAAGPRLFSTVRTVSSHEALSASHILKPGVTSAWIWTRAPTIGGM'
                'RFASTITLGEKTPMKEEDANQKKTENESTGGDAAGGNNKGDKGIASYWGVEPNKITKEDG'
                'SEWKWNCFRPWETYKADITIDLKKHHVPTTFLDRIAYWTVKSLRWPTDLFFQRRYGCRAM'
                'MLETVAAVPGMVGGMLLHCKSLRRFEQSGGWIKALLEEAENERMHLMTFMEVAKPKWYER'
                'ALVITVQGVFFNAYFLGYLISPKFAHRMVGYLEEEAIHSYTEFLKELDKGNIENVPAPAI'
                'AIDYWRLPADATLRDVVMVVRADEAHHRDVNHFASDIHYQGRELKEAPAPIGYH')
Q39219_chain_idx = (63, 354)
Q39219_chain = get_chain(Q39219_fasta, Q39219_chain_idx)

In [69]:
str(Q39219_chain.get_formula())

'C1508H2342N408O389S13'

In [70]:
Q39219_chain.get_mol_wt()

32828.571

In [71]:
Q39219_chain.get_charge()

38

##### Representing the complex with `BcForms`
Then, we can represent the complex with `BcForms`. In particular, the concrete representation of cofactor iron ion can be either achieved by providing a SMILES-encoded string or an `openbabel.OBMol`. Here, we use the SMILES-encoded string for simplicity.

In [72]:
str_procof = ('q39219 + 2 * fe')
bc_form_procof = bcforms.BcForm().from_str(str_procof)
bc_form_procof.set_subunit_attribute('q39219', 'structure', Q39219_chain)
bc_form_procof.set_subunit_attribute('fe', 'structure', '[Fe+2]')

In [73]:
str(bc_form_procof.get_formula())

'C1508Fe2H2342N408O389S13'

In [74]:
bc_form_procof.get_mol_wt()

32940.261000000006

In [75]:
bc_form_procof.get_charge()

42

#### Protein-coenzyme complexes
In this example, we use `BcForms` to represent peroxisomal sarcosine oxidase ([Q9P0Z9](https://www.uniprot.org/uniprot/Q9P0Z9)) in human. The protein is a monomer and binds 1 flavin adenine dinucleotide (FAD) per unit.

##### Representing the subunits with `BpForms`
First, we represent the protein subunit Q9P0Z9 using `BpForms`.

In [76]:
Q9P0Z9_fasta = ('MAAQKDLWDAIVIGAGIQGCFTAYHLAKHRKRILLLEQFFLPHSRGSSHGQSRIIRKAYL'
                'EDFYTRMMHECYQIWAQLEHEAGTQLHRQTGLLLLGMKENQELKTIQANLSRQRVEHQCL'
                'SSEELKQRFPNIRLPRGEVGLLDNSGGVIYAYKALRALQDAIRQLGGIVRDGEKVVEINP'
                'GLLVTVKTTSRSYQAKSLVITAGPWTNQLLRPLGIEMPLQTLRINVCYWREMVPGSYGVS'
                'QAFPCFLWLGLCPHHIYGLPTGEYPGLMKVSYHHGNHADPEERDCPTARTDIGDVQILSS'
                'FVRDHLPDLKPEPAVIESCMYTNTPDEQFILDRHPKYDNIVIGAGFSGHGFKLAPVVGKI'
                'LYELSMKLTPSYDLAPFRISRFPSLGKAHL')
Q9P0Z9_chain_idx = (1, 390)
Q9P0Z9_chain = get_chain(Q9P0Z9_fasta, Q9P0Z9_chain_idx)

In [77]:
str(Q9P0Z9_chain.get_formula())

'C1981H3153N553O515S17'

In [78]:
Q9P0Z9_chain.get_mol_wt()

43502.390999999996

##### Representing the complex with `BcForms`
Then, we can represent the complex with `BcForms`. Again, we will use the SMILES-encoded string to represent the structure of FAD.

In [79]:
str_procoe = ('q9p0z9 + fad')
bc_form_procoe = bcforms.BcForm().from_str(str_procoe)
bc_form_procoe.set_subunit_attribute('q9p0z9', 'structure', Q9P0Z9_chain)
bc_form_procoe.set_subunit_attribute('fad', 'structure', 'c12cc(C)c(C)cc1N=C3C(=O)NC(=O)N=C3N2C[C@H](O)[C@H](O)[C@H](O)COP(=O)(O)OP(=O)(O)OC[C@@H]4[C@@H](O)[C@@H](O)[C@@H](O4)n5cnc6c5ncnc6N')

In [80]:
str(bc_form_procoe.get_formula())

'C2008H3186N562O530P2S17'

In [81]:
bc_form_procoe.get_mol_wt()

44287.947523995994

### Nucleic acid macromolecules with interstrand crosslinks
`BcForms` not only can represent protein complexes but also can concretely describe nucleic acid complexes. This representation is particularly important since interstrand crosslinks in nucleic acid complexes have important biological implications. For example, in DNA, interstrand crosslinks can prevent proper transcription and replication. This mechanism is widely used in chemotherapeutic drugs. For a thorough review on DNA interstrand crosslinks, see [Deans and West, 2011](https://doi.org/10.1038/nrc3088).
Here, we demonstrate `BcForm` representation of DNA interstrand crosslinks using a simple example: cisplatin crosslinking two guanines. Crosslink agent cisplatin is a clinically used drug that induces interstrand crosslinks between two guanines.

##### Representing subunits with `BpForms`
First, we represent the two guanines in the complex using `BpForms`.

In [82]:
guanine = bpforms.DnaForm().from_str('G')

In [83]:
str(guanine.get_formula())

'C10H12N5O7P'

In [84]:
guanine.get_mol_wt()

345.20776199799997

In [85]:
guanine.get_charge()

-2

In [86]:
guanine.export(format='smiles')

'OC1CC(OC1COP(=O)([O-])[O-])n1cnc2c1nc(N)[nH]c2=O'

##### Representing the complex with `BcForms`
We can represent the complex as the subunit composition and the crosslinks.

In [87]:
bc_form_dnaisc = bcforms.BcForm().from_str('2 * g + cisplatin'
                                          '| x-link: [ l-bond-atom: g(1)-1N15'
                                                    '| r-bond-atom: cisplatin-1Pt5'
                                                    '| r-displaced-atom: cisplatin-1Cl6]'
                                          '| x-link: [ l-bond-atom: g(2)-1N15'
                                                    '| r-bond-atom: cisplatin-1Pt5'
                                                    '| r-displaced-atom: cisplatin-1Cl7]')
bc_form_dnaisc.set_subunit_attribute('g', 'structure', guanine)
bc_form_dnaisc.set_subunit_attribute('cisplatin', 'structure', '[NH3+]-[Pt-2](Cl)(Cl)[NH3+]')

In [88]:
bc_form_dnaisc.export()

'OC1CC(OC1COP(=O)([O-])[O-])n1cn(c2c1nc(N)[nH]c2=O)[Pt-2](n1cn(C2CC(O)C(O2)COP(=O)([O-])[O-])c2c1c(=O)[nH]c(n2)N)([NH3+])[NH3+]'

In [89]:
str(bc_form_dnaisc.get_formula())

'C20H30N12O14P2Pt'

In [90]:
bc_form_dnaisc.get_mol_wt()

919.5615239959998

In [91]:
bc_form_dnaisc.get_charge()

-4

### Protein-nucleic acid macromolecules with interstrand crosslinks
In cells, protein-DNA crosslinks can also occur sometimes. There exist multiple mechanisms for their formation. For review, see [Ji et al., 2016](https://doi.org/10.1021%2Facs.accounts.5b00056). Interestingly, some agents that induce DNA-DNA crosslinks, such as cisplatin and nitrogen mustard, can also induce protein-DNA crosslinks. `BcForms` can concretely represent these complexes.
In this example, we show the `BcForms` representation of a simple crosslink between guanine and lysine.

##### Representing subunits with `BpForms`
First, we represent the guanine and the lysine in the complex using `BpForms`.

In [92]:
guanine = bpforms.DnaForm().from_str('G')

In [93]:
guanine.export(format='smiles')

'OC1CC(OC1COP(=O)([O-])[O-])n1cnc2c1nc(N)[nH]c2=O'

In [94]:
lysine = bpforms.ProteinForm().from_str('K')

In [95]:
lysine.export(format='smiles')

'[NH3+]CCCC[C@@H](C(=O)O)[NH3+]'

##### Representing the complex with `BcForms`
Then, we can represent the complex as the subunits and the crosslinks.

In [96]:
bc_form_prodna = bcforms.BcForm().from_str('guanine + cisplatin + lysine'
                                          '| x-link: [ l-bond-atom: guanine-1N15'
                                                    '| r-bond-atom: cisplatin-1Pt5'
                                                    '| r-displaced-atom: cisplatin-1Cl6]'
                                          '| x-link: [ l-bond-atom: lysine-1N1-1' 
                                                    '| r-bond-atom: cisplatin-1Pt5'
                                                    '| l-displaced-atom: lysine-1H1+1'
                                                    '| l-displaced-atom: lysine-1H1'
                                                    '| r-displaced-atom: cisplatin-1Cl7]')
bc_form_prodna.set_subunit_attribute('guanine', 'structure', guanine)
bc_form_prodna.set_subunit_attribute('cisplatin', 'structure', '[NH3+]-[Pt-2](Cl)(Cl)[NH3+]')
bc_form_prodna.set_subunit_attribute('lysine', 'structure', lysine)

In [97]:
bc_form_prodna.export()

'OC1CC(OC1COP(=O)([O-])[O-])n1cn(c2c1nc(N)[nH]c2=O)[Pt-2]([NH3+])([NH3+])NCCCC[C@@H](C(=O)O)[NH3+]'

In [98]:
str(bc_form_prodna.get_formula())

'C16H32N9O9PPt'

In [99]:
bc_form_prodna.get_mol_wt()

720.5437619979998

In [100]:
bc_form_prodna.get_charge()

-1